In [ ]:
# Imports and helper constants
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display

# Import build_daily_report from the reports package
try:
    from reports.daily_report_generator import build_daily_report, OUTPUT_CSV, OUTPUT_MD, METRICS_LOG
except Exception as e:
    # fallback to path-based import
    import importlib.util
    spec_path = os.path.join(os.getcwd(), 'reports', 'daily_report_generator.py')
    try:
        spec = importlib.util.spec_from_file_location('daily_report_generator', spec_path)
        mod = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(mod)
        build_daily_report = getattr(mod, 'build_daily_report')
        OUTPUT_CSV = getattr(mod, 'OUTPUT_CSV')
        OUTPUT_MD = getattr(mod, 'OUTPUT_MD')
        METRICS_LOG = getattr(mod, 'METRICS_LOG')
        print('Imported build_daily_report from', spec_path)
    except Exception as e:
        print('Could not import daily_report_generator:', e)
        build_daily_report = None

sns.set(style='whitegrid')


In [ ]:
# Run the build_daily_report and print results
if build_daily_report is None:
    print('build_daily_report is not available; please check the reports/daily_report_generator.py file')
else:
    # Use the available METRICS_LOG or the default
    metrics_log_path = METRICS_LOG if isinstance(METRICS_LOG, str) else os.path.join(os.getcwd(), 'models', 'metrics_log.csv')
    print('metrics_log_path:', metrics_log_path)
    build_daily_report(metrics_log_path)
    print('Done building daily report (CSV/MD).')


In [ ]:
# Load the generated CSV and display summary
report_csv = OUTPUT_CSV if 'OUTPUT_CSV' in globals() else os.path.join(os.getcwd(), 'reports', 'daily_activity_report.csv')
report_md = OUTPUT_MD if 'OUTPUT_MD' in globals() else os.path.join(os.getcwd(), 'reports', 'daily_activity_report.md')
print('Report CSV:', report_csv)
print('Report MD:', report_md)

if os.path.exists(report_csv):
    df = pd.read_csv(report_csv)
    display(df.head())
    # Parse date
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    # Convert numeric
    for col in ['best_rmse','avg_rmse','n_runs','commits_count']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # Plot best_rmse per day
    if 'best_rmse' in df.columns:
        plt.figure(figsize=(10,4))
        sns.lineplot(data=df.sort_values('date'), x='date', y='best_rmse', marker='o')
        plt.title('Best RMSE per day')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    # Quick table: show days with improvements
    if 'improvement' in df.columns:
        df_imp = df.sort_values('date', ascending=False)
        display(df_imp[['date','n_runs','best_rmse','improvement','per_commit_improvement']].head(10))
else:
    print('Report CSV not found; re-run build_daily_report or check path')


In [ ]:
# Display the markdown report (if present)
if os.path.exists(report_md):
    with open(report_md, encoding='utf-8') as fh:
        md = fh.read()
    display(Markdown(md))
else:
    print('Markdown report not found at', report_md)
